In [18]:
import pandas as pd
import folium
import xml.etree.ElementTree as ET
import os
import cairosvg
import re
import cairosvg
from PIL import Image

In [2]:
df = pd.read_csv('data/df_translated.csv')

In [19]:
df['category'].unique()

array(['Invertebrates and Crustaceans', 'Arthropods', 'Fish', 'Sharks',
       'Freshwater Fish', 'Starfish', 'Rays', 'Electric Fish', 'Other',
       'Seahorses'], dtype=object)

In [20]:
os.listdir('images')

['Invertebrates and Crustaceans.svg',
 '.DS_Store',
 'test',
 'Seahorses.svg',
 'Starfish.svg',
 'Arthropods.svg',
 'Freshwater Fish.svg',
 'Fish.svg',
 'Sharks.svg',
 'Other.svg',
 'Eletric fish.svg',
 'Rays.svg']

In [21]:
df['threat_category'] = df['threat_category'].replace({'Critically Endangered (CR)/Possibly Extinct (PEX)':'Possibly Extinct (PEX)'})
df['acronym_threat_category'] = df['acronym_threat_category'].replace({'CR/PEX':'PEX'})

In [ ]:
df

In [56]:
def convert_svg_to_png_with_color_change(input_svg_file, output_png_file, new_color):
    output_png_file = output_png_file.replace('svg', 'png')
    try:
        # Change the color in the SVG file
        with open(input_svg_file, 'r') as f:
            svg_data = f.read()

        # Replace the fill color with the new color
        new_svg_data = re.sub(
            r'#000000',  # Replace with the old color you want to change
            f'{new_color}',
            svg_data
        )

        # Convert the modified SVG to PNG using cairosvg
        cairosvg.svg2png(bytestring=new_svg_data, write_to=output_png_file)

        print(f"SVG converted to PNG and color changed to {new_color}.")
    except Exception as e:
        print(f"Error: {e}")

In [83]:
def generate_species_info_html(scientific_name, simplified_species, group, threat_category, in_protected_areas,
                            has_conservation_plan, has_fishing_plan, is_exclusive_to_brazil, states, threats):
    protected_status = "present" if in_protected_areas else "absent"
    conservation_plan = "a" if has_conservation_plan else "no"
    fishing_plan = "has" if has_fishing_plan else "has not"
    exclusive_status = "is" if is_exclusive_to_brazil else "is not"

    # Create a list of colored states and threats
    colored_states = [f"<span style='color: #0066BA'>{state}</span>" for state in states]
    colored_threats = [f"<span style='color: red'>{threat}</span>" for threat in threats]

    # Join the colored states and threats with commas
    all_states = ", ".join(colored_states)
    all_threats = ", ".join(colored_threats)
    
    html_text = f"<h1>The {scientific_name}</h1><br>"
    html_text += f"Commonly referred to as {simplified_species}, it belongs to the {group} group. "
    html_text += f"This species is categorized as <span style='color: orange'>{threat_category}</span> and is present in protected areas. "
    html_text += f"There is {conservation_plan} Brazilian conservation action plan in place, and it {fishing_plan} a fishing management strategy. "
    html_text += f"{scientific_name} {exclusive_status} exclusive to Brazil; it can also be found in the states of {all_states}. "
    html_text += f"Its primary threats include {all_threats}."

    return html_text

# Example usage for a real crab species with colored states and threats:
scientific_name = "Ucides cordatus"
simplified_species = "Mangrove crab"
group = "Decapods"
threat_category = "Vulnerable"
in_protected_areas = True
has_conservation_plan = True
has_fishing_plan = False
is_exclusive_to_brazil = False
states = ["Florida", "Louisiana", "Texas", "Brazil", "Argentina"]
threats = ["Habitat loss", "Pollution", "Overharvesting"]

html_info = generate_species_info_html(scientific_name, simplified_species, group, threat_category,
                                    in_protected_areas, has_conservation_plan, has_fishing_plan,
                                    is_exclusive_to_brazil, states, threats)

print(html_info)

<h1>The Ucides cordatus</h1><br>Commonly referred to as Mangrove crab, it belongs to the Decapods group. This species is categorized as <span style='color: orange'>Vulnerable</span> and is present in protected areas. There is a Brazilian conservation action plan in place, and it has not a fishing management strategy. Ucides cordatus is not exclusive to Brazil; it can also be found in the states of <span style='color: #0066BA'>Florida</span>, <span style='color: #0066BA'>Louisiana</span>, <span style='color: #0066BA'>Texas</span>, <span style='color: #0066BA'>Brazil</span>, <span style='color: #0066BA'>Argentina</span>. Its primary threats include <span style='color: red'>Habitat loss</span>, <span style='color: red'>Pollution</span>, <span style='color: red'>Overharvesting</span>.


In [68]:
column_string_to_bool = ['presence_in_protected_areas', 'national_action_plan_for_conservation_', 
                        'fishing_planning','exclusive_species_of_brazil']
string_to_bool_values = {'Yes':True,
                        'No':False}
for i in column_string_to_bool:
    df[i] = df[i].replace(string_to_bool_values)

In [71]:
df[df['commum_name'] == 'Egla']

,group,family,simplified_species,commum_name,threat_category,acronym_threat_category,presence_in_protected_areas,national_action_plan_for_conservation_,fishing_planning,level_of_protection_in_the_national_strategy,exclusive_species_of_brazil,occurrence_states_y,main_threats_y,biome,category
3,Aquatic Invertebrates,Aeglidae,Aegla brevipalma,Egla,Critically Endangered (CR),CR,False,True,False,1.0,True,SC,Fishing,Atlantic Forest,Arthropods
4,Aquatic Invertebrates,Aeglidae,Aegla camargoi,Egla,Endangered (EN),EN,False,True,False,1.0,True,RS,Fishing,Atlantic Forest,Arthropods
5,Aquatic Invertebrates,Aeglidae,Aegla camargoi,Egla,Endangered (EN),EN,False,True,False,1.0,True,SC,Fishing,Atlantic Forest,Arthropods
6,Aquatic Invertebrates,Aeglidae,Aegla cavernicola,Egla,Critically Endangered (CR),CR,True,False,False,2.0,True,SP,Fishing,Atlantic Forest,Arthropods
7,Aquatic Invertebrates,Aeglidae,Aegla franca,Egla,Critically Endangered (CR),CR,False,True,False,1.0,True,MG,Fishing,Atlantic Forest,Arthropods
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,Aquatic Invertebrates,Aeglidae,Aegla strinatii,Egla,Endangered (EN),EN,True,True,False,1.0,True,SP,Tourism,Atlantic Forest,Arthropods
73,Aquatic Invertebrates,Aeglidae,Aegla strinatii,Egla,Endangered (EN),EN,True,True,False,1.0,True,SP,Pollution,Atlantic Forest,Arthropods
74,Aquatic Invertebrates,Aeglidae,Aegla violacea,Egla,Endangered (EN),EN,True,True,False,1.0,True,RS,Fishing,Pampa,Arthropods
75,Aquatic Invertebrates,Aeglidae,Aegla violacea,Egla,Endangered (EN),EN,True,True,False,1.0,True,RS,Tourism,Pampa,Arthropods


In [57]:
colors = ['#581845', '#C70039', '#FF5733', '#FFC300']
dangers = ['PEX', 'CR', 'EN', 'VU']

for svg_file_path in os.listdir('images'):
    for color, danger in zip(colors, dangers):
        convert_svg_to_png_with_color_change(f'images/{svg_file_path}', f'images/test/{danger}-{svg_file_path}', color)

SVG converted to PNG and color changed to #581845.
SVG converted to PNG and color changed to #C70039.
SVG converted to PNG and color changed to #FF5733.
SVG converted to PNG and color changed to #FFC300.
Error: 'utf-8' codec can't decode byte 0xb8 in position 1077: invalid start byte
Error: 'utf-8' codec can't decode byte 0xb8 in position 1077: invalid start byte
Error: 'utf-8' codec can't decode byte 0xb8 in position 1077: invalid start byte
Error: 'utf-8' codec can't decode byte 0xb8 in position 1077: invalid start byte
Error: [Errno 21] Is a directory: 'images/test'
Error: [Errno 21] Is a directory: 'images/test'
Error: [Errno 21] Is a directory: 'images/test'
Error: [Errno 21] Is a directory: 'images/test'
SVG converted to PNG and color changed to #581845.
SVG converted to PNG and color changed to #C70039.
SVG converted to PNG and color changed to #FF5733.
SVG converted to PNG and color changed to #FFC300.
SVG converted to PNG and color changed to #581845.
SVG converted to PNG and 

In [86]:
import folium

# Define the path to your custom icon
icon_path1 = 'images/test/CR-Other.png'
icon_path2 = 'images/test/VU-Seahorses.png'
icon_path3 = 'images/test/PEX-Eletric fish.png'
icon_path4 = 'images/test/EN-Invertebrates and Crustaceans.png'

# Create a custom icon using the SVG file
custom_icon1 = folium.CustomIcon(
    icon_image=icon_path1,
    icon_size=(50, 50),  # Adjust the size of your icon as needed
)

custom_icon2 = folium.CustomIcon(
    icon_image=icon_path2,
    icon_size=(50, 50),  # Adjust the size of your icon as needed
)

custom_icon3 = folium.CustomIcon(
    icon_image=icon_path3,
    icon_size=(50, 50),  # Adjust the size of your icon as needed
)

custom_icon4 = folium.CustomIcon(
    icon_image=icon_path4,
    icon_size=(50, 50),  # Adjust the size of your icon as needed
)
# Create a Folium map
m = folium.Map(location=[-14.235004, -51.92528], zoom_start=4)

# Define a list of marker coordinates and popup texts
marker_data = [
    ([-15.7801, -47.9292], 'Other', custom_icon1),
    ([-16.5122, -42.4993], 'Seahorses', custom_icon2),
    ([-18.5122, -43.6693], 'Eletric Fish', custom_icon3),
    ([-19.5422, -46.1249], "<h1>The Ucides cordatus</h1><br>Commonly referred to as Mangrove crab, it belongs to the Decapods group. This species is categorized as <span style='color: orange'>Vulnerable</span> and is present in protected areas. There is a Brazilian conservation action plan in place, and it has not a fishing management strategy. Ucides cordatus is not exclusive to Brazil; it can also be found in the states of <span style='color: #0066BA'>Florida</span>, <span style='color: #0066BA'>Louisiana</span>, <span style='color: #0066BA'>Texas</span>, <span style='color: #0066BA'>Brazil</span>, <span style='color: #0066BA'>Argentina</span>. Its primary threats include <span style='color: red'>Habitat loss</span>, <span style='color: red'>Pollution</span>, <span style='color: red'>Overharvesting</span>.",custom_icon4),

    # Add more marker coordinates and popup texts here
]

# Add markers to the map
for coords, popup_text, icon in marker_data:
    folium.Marker(location=coords, popup=popup_text, icon=icon).add_to(m)

# Display the map
m.save('demo_map.html') # Save the map as an HTML file or use .show() to display it in your notebook

In [ ]:
# Criar camadas principais (pais)
m = folium.Map(location=[-14.235004, -51.92528], zoom_start=4)


camada_pontos_interesse = folium.FeatureGroup(name='Pontos de Interesse',show=False)
camada_hotel = folium.FeatureGroup(name='Hotéis',show=False)
camada_restaurante = folium.FeatureGroup(name='Restaurantes',show=False)

# Adicione as camadas principais (pais) ao mapa
camada_pontos_interesse.add_to(m)
camada_hotel.add_to(m)
camada_restaurante.add_to(m)

# Adicione camadas secundárias (filhas)
camada_hotel_chique = folium.FeatureGroup(name='Hotéis Chiques', show=False, overlay=True)
camada_hotel_economico = folium.FeatureGroup(name='Hotéis Econômicos', show=False,overlay=True)

for i, j in df.iterrows():
        folium.Marker(
                location=[j['lats_longs_animals'][1], j['lats_longs_animals'][0]],
                popup=j['Nome Comum']
        ).add_to(camada_hotel_chique)

folium.Marker([-22.9068, -43.1729], popup='Hotel de Luxo').add_to(camada_hotel_chique)

# Adicione marcadores para hotéis econômicos
folium.Marker([-15.7801, -47.9292], popup='Hotel Econômico A').add_to(camada_restaurante)
folium.Marker([-22.9068, -43.1729], popup='Hotel Econômico B').add_to(camada_restaurante)

# Adicione as camadas secundárias (filhas) às camadas principais (pais)
camada_hotel_chique.add_to(camada_hotel)
camada_hotel_economico.add_to(camada_hotel)

# Adicione controles de camadas aninhadas (nested layer control)
folium.LayerControl(collapsed=True).add_to(m)

m